In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import json

In [8]:
# Define the data folder path
folder_path = './data/question_1c'
file_type = ".json"

with open(f'{folder_path}/appliance_params{file_type}', 'r') as f:
    appliance_params = json.load(f)

with open(f'{folder_path}/bus_params{file_type}', 'r') as f:
    bus_params = json.load(f)

with open(f'{folder_path}/consumer_params{file_type}', 'r') as f:
    consumer_params = json.load(f)

with open(f'{folder_path}/DER_production{file_type}', 'r') as f:
    DER_production = json.load(f)

with open(f'{folder_path}/usage_preferences{file_type}', 'r') as f:
    usage_preference = json.load(f)


In [9]:
#System Parameters
bus_data = bus_params[0]
tau_imp = bus_data['import_tariff_DKK/kWh']
tau_exp = bus_data['export_tariff_DKK/kWh']
max_import = bus_data['max_import_kW']
max_export = bus_data['max_export_kW']
electricity_prices = bus_data['energy_price_DKK_per_kWh']

In [10]:
#PV parameters
pv_data = appliance_params['DER'][0]
pv_max_power = pv_data['max_power_kW']
pv_profile = DER_production[0]['hourly_profile_ratio']
pv_prod_hourly = [pv_max_power * ratio for ratio in pv_profile] # PV production per hour (kW) (No curtailment)

In [11]:
#Load parameters
load_data = appliance_params['load'][0]
L_max = load_data['max_load_kWh_per_hour'] # Maximum power consumption (kW)
L_ref_ratio = usage_preference[0]['load_preferences'][0]['hourly_profile_ratio'] # Minimum daily consumption (kWh)
L_ref = [L_max * ratio for ratio in L_ref_ratio] # PV production per hour (kW) (No curtailment)

In [12]:
#Battery parameters
Battery_data = appliance_params['storage'][0]
Battery_preferences = usage_preference[0]['storage_preferences'][0]
Batt_cap = Battery_data['storage_capacity_kWh']
Batt_max_ch_power = Battery_data['max_charging_power_ratio']*Batt_cap
Batt_max_dis_power = Battery_data['max_discharging_power_ratio']*Batt_cap
Batt_charging_eff = Battery_data['charging_efficiency']
Batt_discharging_eff = Battery_data['discharging_efficiency']
Batt_initial_soc = Battery_preferences['initial_soc_ratio']*Batt_cap
Batt_final_soc = Battery_preferences['final_soc_ratio']*Batt_cap

In [13]:
#Temporal parameters
T = len(electricity_prices)  # 24 hours
Times = range(T) # Time horizon (24 hours)
# Discomfort parameter
alpha = 17

In [14]:
model = gp.Model("Energy_Optimization")

# Decision variables:
L_t = model.addVars(Times, lb=0, ub=L_max, name="L_t")  # Load consumption (kW)
C_t = model.addVars(Times, lb=0, name="C_t")  # Energy curtailed from PV (kW)
G_imp_t = model.addVars(Times, lb=0, ub=max_import, name="G_imp_t")  # Grid import (kW)
G_exp_t = model.addVars(Times, lb=0, ub=max_export, name="G_exp_t")  # Grid export (kW)
D_t = model.addVars(Times, lb=0, name="D_t")  # Discomfort (kW)
P_batt_ch = model.addVars(Times, lb=0, ub=Batt_max_ch_power, name="P_batt_ch")  # Battery charging power (kW)
P_batt_dis = model.addVars(Times, lb=0, ub=Batt_max_dis_power, name="P_batt_dis")  # Battery discharging power (kW)
SOC = model.addVars(Times, lb=0, ub=Batt_cap, name="SOC")  # State of Charge (kWh)


# Binary variables for mutual exclusivity of import/export
b_imp_t = model.addVars(Times, vtype=GRB.BINARY, name="b_imp_t")  # 1 if importing, 0 otherwise
b_exp_t = model.addVars(Times, vtype=GRB.BINARY, name="b_exp_t")  # 1 if exporting, 0 otherwise

Set parameter Username
Set parameter LicenseID to value 2617496
Academic license - for non-commercial use only - expires 2026-02-03


In [15]:
#Objective function
# Minimize: Import Cost - Export Revenue + Discomfort Penalty
# Import Cost = G_imp * (tau_imp + price)
# Export Revenue = G_exp * (price - tau_exp)
# Discomfort Penalty = alpha * sum(D_t)
model.setObjective(
    gp.quicksum(G_imp_t[t] * (tau_imp + electricity_prices[t]) - 
                G_exp_t[t] * (electricity_prices[t] - tau_exp) for t in Times) + 
    alpha * gp.quicksum(D_t[t] for t in Times),
    GRB.MINIMIZE)

In [23]:
#add constraints

# Curtailment constraint: Cannot curtail more than PV produces
Curtailment_constraint = [
	model.addLConstr(C_t[t] <= pv_prod_hourly[t], name=f"Curtailment_{t}")
	for t in Times
]

# Mutual exclusivity: Cannot import and export simultaneously
Mutual_exclusivity_constraint = [
	model.addLConstr(b_imp_t[t] + b_exp_t[t] <= 1, name=f"Mutual_Exclusivity_{t}")
	for t in Times
]

# Link binary variables to import/export (big-M constraints)
Import_binary_constraint = [
	model.addLConstr(G_imp_t[t] <= max_import * b_imp_t[t], name=f"Import_Binary_{t}")
	for t in Times
]

Export_binary_constraint = [
	model.addLConstr(G_exp_t[t] <= max_export * b_exp_t[t], name=f"Export_Binary_{t}")
	for t in Times
]

# Discomfort constraints: D_t >= |L_t - L_ref[t]|
discomfort_constraint_1 = [
	model.addLConstr(D_t[t] >= L_t[t] - L_ref[t], name=f"discomfort_1_{t}")
	for t in Times
]

discomfort_constraint_2 = [
	model.addLConstr(D_t[t] >= -(L_t[t] - L_ref[t]), name=f"discomfort_2_{t}")
	for t in Times
]

# Power balance constraint
power_balance_constraint = [
	model.addLConstr(G_imp_t[t] - G_exp_t[t] + P_batt_ch[t] - P_batt_dis[t] == L_t[t] - pv_prod_hourly[t] + C_t[t], name=f"power_balance_{t}")
	for t in Times
]

SOC_0 = [
    model.addConstr(SOC[0] == Batt_initial_soc + P_batt_ch[0]*Batt_charging_eff - P_batt_dis[0]/Batt_discharging_eff, name="SOC_0")
]

SOC_dynamics = [
    model.addLConstr(SOC[t] == (SOC[t-1] + P_batt_ch[t]*Batt_charging_eff - P_batt_dis[t]/Batt_discharging_eff), name=f"SOC_dynamics_{t}")
    for t in Times if t > 0
]

SOC_end = [
    model.addLConstr(SOC[T-1] >= Batt_final_soc, name="SOC_end")
]


In [24]:
#solve optimization problem
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 6800HS Creator Edition, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1058 rows, 240 columns and 2640 nonzeros
Model fingerprint: 0x756ad18c
Variable types: 192 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [9e-01, 1e+03]
  Objective range  [4e-01, 2e+01]
  Bounds range     [9e-01, 1e+03]
  RHS range        [1e-01, 3e+00]

MIP start from previous solve produced solution with objective 20.0237 (0.01s)
MIP start from previous solve produced solution with objective 20.0237 (0.01s)
Loaded MIP start from previous solve with objective 20.0237

Presolve removed 916 rows and 36 columns
Presolve time: 0.00s
Presolved: 142 rows, 204 columns, 425 nonzeros
Variable types: 180 continuous, 24 integer (24 binary)

Root relaxation: cutoff, 110 iterations, 0.00 seconds (0.00 work u

In [25]:
#print results
if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Total cost: {model.objVal:.2f} DKK")
    print("Hour | Load (kW) | Curtailment (kW) | Grid Import (kW) | Grid Export (kW) | PV Production (kW) | Electricity Price (DKK/kWh) | State of Charge (kWh) | Battery charging [kW] | Battery discharging [kW]")
    for t in Times:
        print(f"{t:4d} | {L_t[t].X:9.2f} | {C_t[t].X:15.2f} | {G_imp_t[t].X:15.2f} | {G_exp_t[t].X:15.2f} | {pv_prod_hourly[t]:18.2f} | {electricity_prices[t]:25.2f} | {SOC[t].X:18.2f} | {P_batt_ch[t].X:18.2f} | {P_batt_dis[t].X:20.2f}")


Optimal solution found:
Total cost: 20.02 DKK
Hour | Load (kW) | Curtailment (kW) | Grid Import (kW) | Grid Export (kW) | PV Production (kW) | Electricity Price (DKK/kWh) | State of Charge (kWh) | Battery charging [kW] | Battery discharging [kW]
   0 |      0.17 |            0.00 |            0.00 |            0.00 |               0.00 |                      1.10 |               2.99 |               0.90 |                 0.73
   1 |      0.12 |            0.00 |            0.00 |            0.00 |               0.00 |                      1.05 |               2.94 |               0.90 |                 0.78
   2 |      0.12 |            0.00 |            0.00 |            0.00 |               0.00 |                      1.00 |               2.88 |               0.90 |                 0.78
   3 |      0.12 |            0.00 |            0.96 |            0.00 |               0.00 |                      0.90 |               1.76 |               0.90 |                 1.74
   4 |      0